In [ ]:
"""
Broadband Tomography with CASTOR

This script implements a forecast for a broadband tomographic measurement of 
the UV background with the Cosmological Advanced Survey Telescope for 
Optical-UV Research (CASTOR). 

Broadband tomography (arxiv:) applies clustering redshift estimation (arxiv:) to derive
a bias weighted distribution of photon emission that is, in principle, insensitive to 
foreground emission from galactic or solar system components. 

This script is organized as follows. It begins by importing the CASTOR filter curves, 
an emissivity model is then defined, followed by a forward model for the observed frame 
quantity is implemented. With this forward model, given an error model, we then define 
a data likelihood and conclude with an MCMC routine to generate estimates of the 
recovery of the posterior distributions for the input parameters. 

This script can be imported and delivers functions for modeling a broadband tomographic survey. 
Modifying the filter passbands allows one to model other surveys. 

"""

In [1]:
# import statements

import numpy as np
from scipy import interpolate
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt

# Emcee sampling, multiprocessing and analysis
import time
import corner
from multiprocess import Pool

# define background cosmology using the colossus package
# Set to Planck 2018 best fit

from colossus.cosmology import cosmology

cosmo = cosmology.setCosmology('planck18')

# define physical constants 

speed_of_light = 3*10**18 #in angstroms/s
nu1500 = speed_of_light/1500
nu1216 = speed_of_light/1216
nu912 = speed_of_light/912

# define observed frame wavelength and frequency range

obs_waves = np.linspace(300, 6000, 6000-299) #note, this range more than the castor filter set. 

nu_obs = speed_of_light/obs_waves
inverse_nu_obs = 1.0/nu_obs

# import  sampler

import emcee

#set a global path variable to passband files, e.g. for running on a cluster

#path = "/rhome/bscot002/bigdata/passbands/"
path = "passbands/"

In [2]:
# Read in filter curves and interpolate in wavelength and frequency. 

class ThroughputCurve:
    """
    A class representing a throughput curve.

    ...

    Attributes
    ----------
    None

    Methods
    -------
    None 
    """
    
    def __init__(self, dataframe):
        """
        Parameters
        ----------
        waves : pandas dataframe
            The wavelength in microns (galex) or angstroms (castor) which are related by a factor of 1e5
        throughput : pandas dataframe
            The unnormalized filter throughput, e.g. transmission fraction

        """
        self.waves = dataframe[0]
        self.throughput = dataframe[1]

        
# define names of filter curve files
        
castor_names = ['passband_castor_uv.csv', 
                'passband_castor_uv_dark.csv', 
                'passband_castor_u.csv',
                'passband_castor_u_wide.csv', 
                'passband_castor_g.csv']

galex_names = ['galex1500.res.txt', 
               'galex2500.res.txt']

throughput_curve_names = castor_names + galex_names

# and read filter curves to throughput curve object

filter_waves = ThroughputCurve(pd.read_csv(path+castor_names[1], 
                                                      delim_whitespace=True, skiprows=0, header=None)).waves

throughputs = np.zeros((5, 1001))
for i,f in enumerate(castor_names): 
       throughputs[i,:] = ThroughputCurve(pd.read_csv(path+f, 
                                                      delim_whitespace=True, skiprows=0, header=None)).throughput

castor_uv_throughput = throughputs[0,:]
castor_uv_waves = filter_waves

castor_uv = interpolate.interp1d(np.array(castor_uv_waves)*10000, np.array(castor_uv_throughput), bounds_error = False, fill_value = 0)
castor_uv_dark = interpolate.interp1d(np.array(filter_waves)*10000, np.array(throughputs[1,:]), bounds_error = False, fill_value = 0)
castor_u = interpolate.interp1d(np.array(filter_waves)*10000, np.array(throughputs[2,:]), bounds_error = False, fill_value = 0)
castor_u_wide = interpolate.interp1d(np.array(filter_waves)*10000, np.array(throughputs[3,:]), bounds_error = False, fill_value = 0)
castor_g = interpolate.interp1d(np.array(filter_waves)*10000, np.array(throughputs[4,:]), bounds_error = False, fill_value = 0)


throughputs_galex_fuv = ThroughputCurve(pd.read_csv(path+galex_names[0], 
                                                      delim_whitespace=True, skiprows=0, header=None)).throughput
        
filter_waves_fuv = ThroughputCurve(pd.read_csv(path+galex_names[0], 
                                                      delim_whitespace=True, skiprows=0, header=None)).waves

galex_fuv = interpolate.interp1d(np.array(filter_waves_fuv), np.array(throughputs_galex_fuv), bounds_error = False, fill_value = 0)

throughputs_galex_nuv = ThroughputCurve(pd.read_csv(path+galex_names[1], 
                                                      delim_whitespace=True, skiprows=0, header=None)).throughput
        
filter_waves_nuv = ThroughputCurve(pd.read_csv(path+galex_names[1], 
                                                      delim_whitespace=True, skiprows=0, header=None)).waves

galex_nuv = interpolate.interp1d(np.array(filter_waves_nuv), np.array(throughputs_galex_nuv), bounds_error = False, fill_value = 0)

filters = np.array([castor_uv, castor_uv_dark, castor_u, castor_u_wide, castor_g, galex_fuv, galex_nuv])

In [4]:
# this cell initializes the parameter vector for later
# use with the MCMC routine

log_e1500z0_b1500z0 = 25.13  # note this degeneracy needs to be resolved
gammae1500 = 2.06
alpha1500z0 = -0.08
Calpha1500 = 1.85
alpha1100z0 = -3.71
Calpha1100 = 0.50 
EWLyAz03 = -6.17
EWLyAz1 = 88.02
gammabnu = -0.86
gammabz = 0.79

#normalizations

b1500z0 = 0.32 #from "breaking the degeneracy section" --> follow up on this! 
e1500z0 = (10**(log_e1500z0_b1500z0))/b1500z0

#assumed values

alpha900 = -1.5
logfLyCz1 = -0.53 
logfLyCz2 = -0.84 

fLyCz1 = np.exp(logfLyCz1)
fLyCz2 = np.exp(logfLyCz2)

theta =  gammabnu, gammabz, log_e1500z0_b1500z0, gammae1500, alpha1500z0, Calpha1500, alpha1100z0, Calpha1100, EWLyAz1, logfLyCz1, logfLyCz2

In [ ]:
def CLyC(logfLyCz2 = -0.84, logfLyCz1 = -0.53):
    """Returns CLyC parameter used to model the evolution of the LyC escape fraction. 

    Parameters
    ----------
    logfLyCz2 : log10 of the LyC escape fraction at z=2, fiducial = -0.84
    logfLyCz1 : log10 of the LyC escape fraction at z=1, fiducial = -0.53

    Returns
    -------
    float
        the value of the CLyC parameter used to evaluate equation 7
    """
    return (logfLyCz2 - logfLyCz1)/np.log10((1+2)/(1+1))

# def logfLyC(z, CLyC, fLyCz2, fLyCz1):
#     return CLyC(fLyCz2, fLyCz1) * np.log10((1+z)/(1+1)) + np.log10(fLyCz2)

def logfLyC(z, logfLyCz2 = -0.84, logfLyCz1 = -0.53):
    """ Returns the log of the Lyman-Continuum escape fraction evolutions with redshift 
    as a function of observed values at redshifts 1 and 2.

    Parameters
    ----------
    z : The emission redshift 
    logfLyCz2 : log10 of the LyC escape fraction at z=2, fiducial = -0.84
    logfLyCz1 : log10 of the LyC escape fraction at z=1, fiducial = -0.53

    Returns
    -------
    float
        The value of the f_LyC parameter defined in equation 7
    """

    return CLyC(logfLyCz2, logfLyCz1) * np.log10((1+z)/(1+1)) + logfLyCz2

#Lyman alpha parameters 

def CLyA(EWLyAz1 = 88.02, EWLyAz03 = -6.17):
    
    """ Returns the log of the Lyman-Continuum escape fraction evolutions with redshift 
    as a function of observed values at redshifts 1 and 2.

    Parameters
    ----------
    EWLyAz1 : The equivalent width of Ly-alpha at z = 1, fiducial = 88.02
    EWLyA03 : The equivalent width of Ly-alpha at z = 0.3, fiducial = -6.17

    Returns
    -------
    float
        the value of the CLyA parameter defined in order to evaluate equation 10
    """
        
    return (EWLyAz1 - EWLyAz03)/np.log10((1+1)/(1+0.3))

def EWLyA(z, EWLyAz1 = 88.02, EWLyAz03 = -6.17):
    """ Returns the equivalent width of Lyman-alpha as a function of redshift

    Parameters
    ----------
    z : The emission redshift
    EWLyAz1 : The equivalent width of Ly-alpha at z = 1, fiducial = 88.02
    EWLyA03 : The equivalent width of Ly-alpha at z = 0.3, fiducial = -6.17

    Returns
    -------
    float
        the value of the Lyman alpha equivalent width as a function of redshift, see equation 10
    """
        
    return CLyA(EWLyAz1, EWLyAz03) * np.log10((1+z)/(1+0.3)) + EWLyAz03

def e1500_evol(z, gammae1500 = 2.06):
    """ Returns evolution of the 1500 Angstrom emissivity as a function of redshift. 

    Parameters
    ----------
    z : The emission redshift
    gammae1500 : The (power law) evolution of the emission with redshift, fiducial = 2.06

    Returns
    -------
    float
        The evolution of the emission at 1500 Angstrom as a function of redshift as defined in equation 12. 
    """
    
    return (1+z)**(gammae1500)

def alpha1500(z, Calpha1500=1.85, alpha1500z0=-0.08):
    """ Returns evolution of the 1500 Angstrom slope as a function of redshift. 

    Parameters
    ----------
    z : The emission redshift
    alpha1500z0 : The (power law) evolution of the emission with redshift, fiducial = -0.08
    Calpha1500 : Normalization of the evolution with redshift, fiducial = 1.85
    
    Returns
    -------
    float
        The evolution of the slope at 1500 Angstrom as a function of redshift as defined in equation 8. 
    """
    
    return alpha1500z0 + Calpha1500*np.log10(1+z)

def alpha1100(z, Calpha1100 = 0.50, alpha1100z0 = -3.71):
    """ Returns evolution of the 1100 Angstrom slope as a function of redshift. 

    Parameters
    ----------
    z : The emission redshift
    alpha1100z0 : The (power law) evolution of the emission with redshift, fiducial = -3.71
    Calpha1100 : Normalization of the evolution with redshift, fiducial = 0.50
    
    Returns
    -------
    float
        The evolution of the slope at 1100 Angstrom as a function of redshift as defined in equation 8. 
    """
    
    #switched log10 --> log
    return alpha1100z0 + Calpha1100*np.log10(1+z)

In [ ]:
# begin by interpolating the filter curves to this wavelength range

#nuv

throughputs_galex_nuv = ThroughputCurve(pd.read_csv(path+galex_names[1], 
                                                      delim_whitespace=True, skiprows=0, header=None)).throughput
        
filter_waves_nuv = ThroughputCurve(pd.read_csv(path+galex_names[1], 
                                                      delim_whitespace=True, skiprows=0, header=None)).waves 

#filter waves is in angstroms, speed of light in angstroms/second, so this is in Hz 

filter_freqs_nuv = speed_of_light/filter_waves_nuv 

#interpolate

nuv_freq = interpolate.interp1d(filter_freqs_nuv, throughputs_galex_nuv, bounds_error = False, fill_value = 0)

#and for fuv

throughputs_galex_fuv = ThroughputCurve(pd.read_csv(path+galex_names[0], 
                                                      delim_whitespace=True, skiprows=0, header=None)).throughput
        
filter_waves_fuv = ThroughputCurve(pd.read_csv(path+galex_names[0], 
                                                      delim_whitespace=True, skiprows=0, header=None)).waves

filter_freqs_fuv = speed_of_light/filter_waves_fuv 

fuv_freq = interpolate.interp1d(filter_freqs_fuv, throughputs_galex_fuv, bounds_error = False, fill_value = 0)

filter_freqs = speed_of_light/(filter_waves*10000) # to convert from angstroms

# interpolating functions for filter curves

castor_uv = interpolate.interp1d(np.array(filter_freqs), np.array(castor_uv_throughput), bounds_error = False, fill_value = 0)
castor_u = interpolate.interp1d(np.array(filter_freqs), np.array(throughputs[2,:]), bounds_error = False, fill_value = 0)
castor_g = interpolate.interp1d(np.array(filter_freqs), np.array(throughputs[4,:]), bounds_error = False, fill_value = 0)

In [ ]:
def b(theta, nu, z):
    """
    Returns evolution of the intensity map bias, b_im as a function of redshift and frequency 
    as described in equation 17. 

    Parameters
    ----------
    z : The emission redshift
    nu: frequency in Hz
    theta: A vector of ordered parameters expected by the MCMC routine, 
    theta[0] should be the power law evolution in frequency
    theta[1] should be the power law evolution in redshift 
    
    Returns
    -------
    float
        The (unnormalized) evolution of the map bias in frequency and redshift 
    """
    
    gammabnu = theta[0]
    gammabz = theta[1]
    
    return (nu/nu1500)**(gammabnu) * (1+z)**(gammabz)

def delta(nu):
    """
    For a vector of frequencies, returns a value 1 at the frequency corresponding 
    to 1216 angstroms and 0 elsewhere.
    
    Note
    ----------
    no longer used. 
    """
    condlist = []
    funclist = [1, 0]
    return np.select(condlist, funclist)

def prefactor(z): 
    """
    Returns the multiplicative pre-factor in the observed frame intensity forward model. 
    This is essentially the luminosity distance.
    Calls and requires the colossus cosmology to be set globally. Colossus is a convenience package of cosmology 
    functions.  

    Parameters
    ----------
    z : The emission redshift
    
    Returns
    -------
    float
        The (unnormalized) evolution of the map bias in frequency and redshift 
        
    Requires
    --------
    Calls and requires the colossus cosmology to be set globally. Colossus is a convenience package of cosmology 
    functions.  
    
    """
        
    MPCtoCM = 3.086e24
    speed_of_light_km_s = 3*10**5
    return speed_of_light_km_s *(10**23)/(4*np.pi*(cosmology.Cosmology.Hz(cosmo, z)*(1+z))*(MPCtoCM**2))

def optical_depth(nu, z):
    """
    An analytic approximation to effective optical depth for Poisson distributed absorbers 
    to redshift z from Madau (2000) 

    Parameters
    ----------
    z : The emission redshift
    
    Returns
    -------
    float
        The effective optical depth.  
    
    """
    
    
    A = 4*10**7
    sigma_L = 6.3*10**(-18)
    prefactor = (4.0/3.0)*np.sqrt(np.pi * sigma_L)
    nu_L = 3.29 * 10**15 #Hz, Rydberg frequency
    nu_term = (nu/nu_L)**(-1.5)
    
    #note redshift term assumes z0 = 0
    
    redshift_term = (1**(1.5))*((1+z)**(1.5)-(1)**(1.5))
    
    return np.exp(-prefactor * A * nu_term * redshift_term)


def emissivity_model_optical_depth(theta, nu, z):
    """
    Returns a vector of emissivities*bias*optical depth for a set of input frequencies at a given redshift z. This implements
    the model described in Section 2 and multiplies by the optical depth as described in the forward model given in
    equation 16.

    Parameters
    ----------
    theta : a vector of input parameters describing the emissivity model
    the parameters of theta, in order, are;  
    the evolution of the bias in redshift and frequency,
    the log10(normalization) of the 1500 angstrom emissivity times the bias,
    the power law evolution parameter of the 1500 angstrom emissivity,
    the power law evolution and amplitude of the slope as measured at redshift 0,
    the same quantities at 1100 angstroms,
    the lyman alpha equivalent width at z=1,
    and the log10(LyC) escape fractions at z=1,2. 
    nu: a vector of frequencies to evaluate the emissivity over. The frequencies should correspond to 
    the CASTOR observed wavelength range
    z : The emission redshift
    
    Returns
    -------
    np.array 
        The bias weighted emissivity as a function of frequency at redshift z times the optical depth to redshift z. 
    
    """
    
    
    gammabnu, gammabz, log_e1500z0_b1500z0, gammae1500, alpha1500z0, Calpha1500, alpha1100z0, Calpha1100, EWLyAz1, logfLyCz1, logfLyCz2 = theta
    
    # note, we use np.select to vectorize our piecewise function in frequency. 
    
    condlist = [
                speed_of_light/nu > 1216.5,
                (1215.5 > (speed_of_light/nu)) * (912 <= speed_of_light/nu),
                (np.trunc(speed_of_light/nu)) >= 1215.5 * ((np.trunc(speed_of_light/nu)) <= 1216.5), 
                speed_of_light/nu < 912
                ]
    funclist = [
                10**(log_e1500z0_b1500z0) * e1500_evol(z, gammae1500) * (nu/nu1500)**(alpha1500(z, Calpha1500, alpha1500z0)), 
        
                10**(log_e1500z0_b1500z0) * (e1500_evol(z, gammae1500) *  (nu1216/nu1500)**(alpha1500(z, Calpha1500, alpha1500z0))*
                ((nu/nu1216)**alpha1100(z, Calpha1100, alpha1100z0)))*optical_depth(nu, z), 
        
                10**(log_e1500z0_b1500z0) * (e1500_evol(z, gammae1500) *  (nu1216/nu1500)**(alpha1500(z, Calpha1500, alpha1500z0))*
                ((nu/nu1216)**alpha1100(z, Calpha1100, alpha1100z0) + EWLyA(z, EWLyAz1, EWLyAz03)))*optical_depth(nu, z), 
                #(nu**2)/speed_of_light),
        
                10**(logfLyC(z, logfLyCz2, logfLyCz1))*
                10**(log_e1500z0_b1500z0) * e1500_evol(z, gammae1500) * (nu1216/nu1500)**alpha1500(z, Calpha1500, alpha1500z0) * 
                (nu912/nu1216)**alpha1100(z, Calpha1100, alpha1100z0) * (nu/nu912)**alpha900
                ] 
    return np.select(condlist, funclist)

def biased_weighted_emissivity(theta, z):
    """
    Returns the observed frame quantity dJ/dz b_im as described in equation 16. Note that the name of this function is a
    misnomer and retained for consistency. This is a bias weighted filter specific intensity that is seen in the observed
    frame, while the emissivity is the rest frame underlying quantity whose model we seek to infer. 

    Parameters
    ----------
    theta : a vector of input parameters describing the emissivity model, see documentation for the emissivity model
    for details on this vector. 
    z : The emission redshift
    
    Returns
    -------
    np.array 
        A 3xN vector of the observed frame quantity dJ/dz b_im for each castor filter. 
    
    """
    
    # compute each term in this integral, nu_obs is defined globally
    inverse_nu_obs = 1.0/nu_obs 
    
    #compute the bandpass
    
    R_castor_u = np.abs(np.trapz(castor_u(nu_obs)*inverse_nu_obs, nu_obs))
    R_castor_uv = np.abs(np.trapz(castor_uv(nu_obs)*inverse_nu_obs, nu_obs))
    R_castor_g = np.abs(np.trapz(castor_g(nu_obs)*inverse_nu_obs, nu_obs))
    
    # in this case, the galex nuv curve R_nuv (nu_obs)
    
    R_nu_obs_u = castor_u(nu_obs)
    R_nu_obs_uv = castor_uv(nu_obs)
    R_nu_obs_g = castor_g(nu_obs)

    # the bias factor, evaluated in the rest frame nu = nu_obs (1+z)
    
    bias_nu = b(theta, nu_obs * (1+z), z)
    
    # and the emissivity model itself 
    
    e_nu_z = emissivity_model_optical_depth(theta, nu_obs * (1+z), z)
    
    # and perform the integral
    #syntax for np.trapz is (f(x), x)
    
    dJdz_bj_u = prefactor(z)/R_castor_u * np.abs(np.trapz(R_nu_obs_u * bias_nu * e_nu_z *inverse_nu_obs, nu_obs))
    dJdz_bj_uv = prefactor(z)/R_castor_uv * np.abs(np.trapz(R_nu_obs_uv * bias_nu * e_nu_z *inverse_nu_obs, nu_obs))
    dJdz_bj_g = prefactor(z)/R_castor_g * np.abs(np.trapz(R_nu_obs_g * bias_nu * e_nu_z *inverse_nu_obs, nu_obs))
    

    return np.array([dJdz_bj_uv, dJdz_bj_u, dJdz_bj_g])

In [ ]:
def frac_shot_error(depth = 1):
    """
    Returns the fractional shot noise error for the surveys we consider. 
    The shot noise term is computed separately from a collection of survey catalogs and loaded as a .npy
    This implements the denominator of equation 22, where the full variance is determined in quadrature.  

    Parameters
    ----------
    depth : a factor to multiply the suvey by in order to determine an optimal survey. By default this is 1 to use
    the real survey catalog. 
    
    Returns
    -------
    np.array 
        A 80x1 vector of shot noise terms for each redshift bin we consider in this work.  
    
    """
    
    survey_shot = np.load('frac_errors_survey.npy')
    return (1/depth)*survey_shot

def photo_z_error(A_n, zs):
    """
    Returns the fractional error due to photometric noise as it evolves with redshift as defined in equation 26. 

    Parameters
    ----------
    A_n : The overall noise amplitude. Typical values are 0.01 for CASTOR, 0.03 for GALEX. 
    zs : redshifts to evaluate the noise at it. 
    
    Returns
    -------
    np.array 
        A 80x1 vector of fractional photometric zero-point errors for the redshift binning we consider in this work.  
    
    """
    
    return A_n*(1+zs)**2

def bias_error(sigma_z, alpha, zs):
    """
    Returns the fractional error due to unmodeled evolution of the reference catalog bias. 
    This implements equation 25. 

    Parameters
    ----------
    sigma_z : gaussian width of the tracer catalog redshift distribution, we assume z = 0.5
    alpha : power law describing the unmodeled evolution of the bias. 
    zs : redshifts to evaluate the noise at it. 
    
    Returns
    -------
    np.array 
        A 80x1 vector of fractional bias errors for the redshift binning we consider in this work.  
    
    """
    
    
    error = []

    for z0 in zs:
        weights = norm.pdf(zs, z0, sigma_z)
        error.append((np.average(zs, weights = (zs**alpha)*weights) - np.average(zs, weights = weights)))
    return np.array(error)

def tot_frac_error(zs, depth, model):
    """
    Computes the total fractional error on the observed frame quantity dJ/dz b_im by adding in quadruture.  

    Parameters
    ----------
    zs : redshifts to evaluate the noise at it. 
    depth : survey depth factor to scale the shot noise by
    model : integer selecting either model 1, the optimal model which is photometric error dominated
            or model 2, the full model incoroporating all effects above. 
    
    Returns
    -------
    np.array 
        A 80x1 vector of fractional errors for the redshift binning we consider in this work.  
    
    """
    
    if model == 1:
        A = np.sqrt(frac_shot_error(5)**2 + 0.01**2)
        
    elif model == 2:
        A = np.sqrt(frac_shot_error(1)**2 + photo_z_error(0.01, zs)**2 + bias_error(0.5, 0.01, zs)**2)
        
    return A



def mk_data(theta, zmin = 0.01, zmax = 4, sample_size = 80, depth = 1, error_model = 1):  
    
    """
    This function makes the final data and error vectors.   

    Parameters
    ----------
    theta : a vector of input parameters describing the emissivity model, see documentation for the emissivity model
    for details on this vector.  
    zmin : minimum redshift to compute model over, fiducial = 0.01, should be nonzero and positive
    zmax : maximum redshift to compute model over, fiducial = 4, should be nonzero and positive
    sample_size : number of redshift bins to use. fiducial is 80, corresponding to z_step = 0.05
    error_model : parameter specifying which error model to use. 1 or 2, 1 corresponds to an optimal model, 
                  2 to the full model. 
    
    Returns
    -------
    np.array 
        sim_data is the simulated data under our SED model evaluated at the bin spacing
    np.array
        error is the uncertainty on the sim data, computed as the fractional error * sim_data 
        and evaluated at the bin spacing
    
    """
    
    z_int_sampling = np.linspace(zmin, zmax, sample_size)
    sim_data = np.zeros((len(z_int_sampling), 3))
    error = np.zeros((len(z_int_sampling), 3))
    
    for i, z in enumerate(z_int_sampling):
        sim_data[i,:] = biased_weighted_emissivity(theta, z)
        error[i,:] = sim_data[i,:]*tot_frac_error(z_int_sampling, depth, error_model)[i] #(z, n_galaxies, n_fields)
        #dJdzb_error = np.random.multivariate_normal(mean=np.array([0, 0, 0]), cov=np.diag(error[i,:]))
        #sim_data[i,:] = dJdzb + dJdzb_error

    return sim_data, error

def model(theta, z):
    """
    Convenience wrapper for the biased_weighted_emissivity. See documentation for that function. 
    """
    return biased_weighted_emissivity(theta, z)

In [ ]:
def lnprior(theta):
    """
    This function returns the log of the prior on each parameter as summarized in Table 1.
    
    Parameters
    ----------
    theta : a vector of input parameters describing the emissivity model, see documentation for the emissivity model
    for details on this vector.  
    
    Returns
    -------
    np.array
        log(prior)
    
    """
    
    gammabnu, gammabz,log_e1500z0_b1500z0, gammae1500, alpha1500z0, Calpha1500, alpha1100z0, Calpha1100, EWLyAz1, logfLyCz2, logfLyCz2 = theta

    
    condlist = [-0.08 - (2*0.84)<=alpha1500z0<=-0.08+(2*1.28), 1.85-(2*1.28)<=Calpha1500<=1.85+(2*1.22),
               -3.71 - (2*0.98)<=alpha1100z0<=-3.71 + (2*1.34), 0.50 - (2*1.44)<=Calpha1100<=0.50+(2*1.46),
                88.02 - (2*48.87) <= EWLyAz1 <= 88.02 + (2*51.44), 
                logfLyCz1 <= 0.0, logfLyCz2 <= 0.0, 
               -0.86 - (2*1.29) <= gammabnu<= -0.86 + (2*0.83)]
    
    if not(all(condlist)):
        return -np.inf
    
    mu1 = 2
    sigma1 = 0.3
    mu2 = 0.79
    sigma2 = 0.33
    mu3 = 25.13
    sigma3 = 0.02
    
    return (np.log(1.0/(np.sqrt(2*np.pi)*sigma1))-0.5*(gammae1500-mu1)**2/sigma1**2)+(np.log(1.0/(np.sqrt(2*np.pi)*sigma2))-0.5*(gammabz-mu2)**2/sigma2**2)+(np.log(1.0/(np.sqrt(2*np.pi)*sigma3))-0.5*(log_e1500z0_b1500z0-mu3)**2/sigma3**2)


def lnlike_parallel(theta, z, datum, datum_error):
    """
    This function returns the log likelihood of the data given the model and is parallelized for use with the 
    emcee parallel sampler. 
    
    Parameters
    ----------
    theta : a vector of input parameters describing the emissivity model, see documentation for the emissivity model
    for details on this vector.
    z : redshift to compute the likelihood at
    datum : individual simulated data point corresponding to the redshift z
    datum_error : individual simulated data error corresponding to the data at redshift z
    
    Returns
    -------
    float
        log(like)
    """
    
    diff = model(theta, z) - datum
    icov = np.diag(np.array([1/datum_error[0]**2, 1/datum_error[1]**2, 1/datum_error[2]**2]))
    
    return -0.5* (np.dot(np.transpose(diff), np.dot(icov, diff)))


def lnprob_parallel(theta, zs, data):
    """
    This function returns the log posterior probability of the data given the model 
    and is parallelized for use with the emcee implementation of a parallel sampler. 
    
    Parameters
    ----------
    theta : a vector of input parameters describing the emissivity model, see documentation for the emissivity model
    for details on this vector.
    zs : redshift to evaluate at
    data : data and errors computed from the make_data function. 
    
    Returns
    -------
    float
        log(posterior)
    
    """
    
    lp = lnprior(theta)
    
    lks = np.zeros(len(zs))
    
    for i, (z, datum, datum_error) in enumerate(zip(zs, data[0], data[1])):
        lks[i] = lnlike_parallel(theta, z, datum, datum_error)
    lksum = np.sum(lks)
    if not np.isfinite(lp):
        return -np.inf
    if not np.isfinite(lksum):
        return -np.inf
    return lp + lksum



def main():
    """
    main function of the script, sets and initializes various parameters, prints files to log and calls emcee 
    sampler as well as saving example corner plot locally. 
    """
    
    
    print('Starting emcee sampling')
    nwalkers = 100
    ndim = len(theta)
    n_samps = 80
    chain_len = 10**5
    zmax = 4
    zmin = 0.01
    zs = np.linspace(zmin,zmax, n_samps) #define redshift sampling
    print('nwalkers, ndim, n_samps, chain_len, min(zs), max(zs)')
    print([nwalkers, ndim, n_samps, chain_len, min(zs), max(zs)])
    
    print('Initializing Walkers')

    p0 = theta + 1e-3 * np.random.randn(nwalkers, len(theta))
    
    print('Making data')

    data = mk_data(theta, zmin = zmin, zmax = zmax, sample_size = n_samps, error_model = 1)
    
    print(np.min(data[0]), np.max(data[0]), np.mean(data[0]), np.min(data[1]), np.max(data[1]), np.mean(data[1]))
    
    from multiprocess import cpu_count

    ncpu = cpu_count()
    print("{0} CPUs".format(ncpu))
    
    filename = "/rhome/bscot002/bigdata/error_model_1_1223_optimal.h5"
    backend = emcee.backends.HDFBackend(filename)
    #backend.reset(nwalkers, ndim)

    with Pool() as pool: 
        sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob_parallel, args = [zs, data], pool=pool, backend = backend)
        #sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob_parallel, args = [zs, data], pool=pool)
        start = time.time()
        pos, prob, state = sampler.run_mcmc(p0, chain_len, progress = True)
        end = time.time()
        multi_time = end - start
        print("Multiprocessing took {0:.1f} seconds".format(multi_time))
        
    tau = sampler.get_autocorr_time()
    print('sampling complete')
    print('tau is:')
    print(tau)
    
    flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
    print(flat_samples.shape)
    
    fig = corner.corner(flat_samples);
    
    fig.savefig(path+'corner_test.png')
        
    pass

if __name__ == "__main__":
    main()


